# ToDo :

* ~~consolidate code from notebook into WISDM helper methods file~~
* ~~rewrite kfolds process so that we have more precise control over the size of a fold (as oppose to just the number of folds)~~
* ~~run experiment that directly compares model using ALL general data + active data to model using ONLY general data from nearest cluster + active data~~
* analyze/visualize clusters (is there a better algorithm? is there a better k for the k-means?) 
* perhaps compare with using the WORST cluster, or using ONLY the personal data
    * for each size of active data
        * for each algorithm (also ensemble algorithm?)
            * personal only
            * universal only
            * personal + ALL universal
            * personal + best cluster universal
            * personal + worst cluster universal
* Run experiment trained on v1.1 applied to v2.0 data


In [3]:
from wisdm import wisdm
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.cluster import KMeans
from scipy.stats import mode

import warnings
import time

In [4]:
wisdm.set_data(version='2', make_compatible=True)

# Helpful links in understanding model uncertainty
* [stats.stackexchange.com Random Forest Probabilistic Prediction vs majority vote](https://stats.stackexchange.com/questions/127077/random-forest-probabilistic-prediction-vs-majority-vote)
* [Scikit-Learn Probability calibration of classifiers](http://scikit-learn.org/stable/auto_examples/calibration/plot_calibration.html)

# Calibrate probability of RFC

In [5]:
wisdm.set_data(version="1")

In [35]:
# Train model with v1.1 data and get clusterings
wisdm.set_data(version='1', make_compatible=True)

data_df_v1 = wisdm.remove_all_nan(wisdm.data_df)
user_ids_v1 = wisdm.user_ids

impersonal_labels = np.array([t.decode("utf-8") for t in data_df_v1['class'].as_matrix()])
impersonal_features = data_df_v1.as_matrix(columns=[data_df_v1.columns[1:-1]])

# train an impersonal model
impersonal_scaler = StandardScaler().fit(impersonal_features)
scaled_train_x = impersonal_scaler.transform(impersonal_features)

rfc_clf = wisdm.weka_RF()
rfc_clf.fit(scaled_train_x, impersonal_labels)

sig_clf = CalibratedClassifierCV(rfc_clf, method='sigmoid')
sig_clf.fit(scaled_train_x, impersonal_labels)
    

CalibratedClassifierCV(base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=6,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
            cv=3, method='sigmoid')

In [36]:
# reset data back to v2.0
wisdm.set_data(version="2", make_compatible=True)

In [37]:
user_id = wisdm.user_ids[0]
personal_set = wisdm.get_user_set(user_id)
personal_set = wisdm.remove_all_nan(personal_set)

personal_set = personal_set.sample(frac=1).reset_index(drop=True)
print("%s personal samples" % len(personal_set))

168 personal samples


In [38]:
personal_labels = np.array([t.decode("utf-8") for t in personal_set['class'].as_matrix()])
personal_features = personal_set.as_matrix(columns=[personal_set.columns[1:-1]])

In [39]:
personal_labels[:10]

array(['Standing', 'Walking', 'Jogging', 'Walking', 'Standing', 'Stairs',
       'Walking', 'Standing', 'Sitting', 'Standing'],
      dtype='<U8')

In [40]:
sig_clf.predict(personal_features[:10])

array(['Walking', 'Jogging', 'Jogging', 'Jogging', 'Walking', 'Jogging',
       'Jogging', 'Walking', 'Walking', 'Walking'],
      dtype='<U8')

In [42]:
sig_clf.score(personal_features[:10], personal_labels[:10])

0.10000000000000001

In [69]:
sig_clf.classes_

array(['Jogging', 'Sitting', 'Stairs', 'Standing', 'Walking'],
      dtype='<U8')

In [41]:
rfc_clf.score(personal_features[:10], personal_labels[:10])

0.10000000000000001

In [43]:
probas = sig_clf.predict_proba(personal_features[:10])
probas

array([[ 0.03512588,  0.00250848,  0.40460312,  0.00243261,  0.55532991],
       [ 0.85914587,  0.00229369,  0.03958215,  0.00231072,  0.09666757],
       [ 0.91665366,  0.00232164,  0.02305344,  0.00231334,  0.05565792],
       [ 0.83551717,  0.00226105,  0.03787848,  0.00227902,  0.12206428],
       [ 0.01980574,  0.002398  ,  0.30631995,  0.00236603,  0.66911027],
       [ 0.82258913,  0.00225038,  0.04096054,  0.00227173,  0.13192822],
       [ 0.87401216,  0.00232057,  0.03844194,  0.00233522,  0.08289011],
       [ 0.03506316,  0.00250287,  0.39766928,  0.0024277 ,  0.56233699],
       [ 0.0473069 ,  0.00653889,  0.35332182,  0.00271482,  0.59011758],
       [ 0.03514493,  0.00250925,  0.40035893,  0.00243398,  0.5595529 ]])

In [62]:
predictions = sig_clf.predict(personal_features)
probas = sig_clf.predict_proba(personal_features)

In [63]:
correct_ind=[]
wrong_ind = []

for ind, pred in enumerate(predictions):
    if pred == personal_labels[ind]:
        correct_ind.append(ind)
    else:
        wrong_ind.append(ind)

In [64]:
len(correct_ind) + len(wrong_ind)

168

In [65]:
len(correct_ind)

19

In [66]:
correct_probas = [probas[ind] for ind in correct_ind]
wrong_probas = [probas[ind] for ind in wrong_ind]

In [70]:
probas.argmax(axis=1)

array([4, 0, 0, 0, 4, 0, 0, 4, 4, 4, 4, 0, 4, 4, 4, 4, 0, 4, 4, 0, 0, 0, 4,
       4, 4, 4, 4, 4, 4, 0, 0, 4, 4, 0, 0, 4, 4, 0, 0, 4, 0, 0, 0, 4, 0, 4,
       0, 2, 4, 0, 0, 0, 0, 4, 0, 4, 0, 0, 4, 4, 0, 0, 0, 4, 0, 0, 0, 4, 2,
       0, 4, 0, 4, 0, 0, 0, 0, 4, 0, 4, 0, 4, 0, 0, 0, 0, 4, 0, 4, 0, 4, 4,
       0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 4, 0, 2, 4, 4, 0, 0, 4, 4, 0, 0, 4, 4,
       0, 4, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 4, 0, 0, 0, 4, 4, 4, 4, 4, 0, 0, 4, 0, 4, 4, 0, 0, 0, 0,
       4, 4, 0, 4, 4, 0, 4])

In [67]:
np.mean([np.max(i) for i in correct_probas])

0.91489408066801359

In [68]:
np.mean([np.max(i) for i in wrong_probas])

0.722228846423148

In [48]:
len(personal_features)

168

In [20]:
sig_clf.classes_

array(['Jogging', 'Sitting', 'Stairs', 'Standing', 'Walking'],
      dtype='<U8')

In [9]:
from sklearn.metrics import brier_score_loss
from sklearn.calibration import CalibratedClassifierCV

# hold out a random set of users for testing
test_user_inds = np.random.choice(len(wisdm.user_ids), 10)
test_users = [wisdm.user_ids[i] for i in test_user_inds]
test_set = pd.concat([wisdm.get_user_set(user_id) for user_id in test_users])

# 

In [10]:
test_set.describe()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,XPEAK,YPEAK,ZPEAK,XABSOLDEV,YABSOLDEV,ZABSOLDEV,XSTANDDEV,YSTANDDEV,ZSTANDDEV,RESULTANT
count,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.00000,1337.000000,1337.000000,1337.000000,1337.000000,...,1240.000000,1286.000000,1328.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000
mean,0.097270,0.097210,0.099761,0.095079,0.096013,0.10377,0.095999,0.105221,0.098010,0.088901,...,1908.426250,1178.073919,1967.210437,4.771735,3.815475,2.933747,5.838616,4.838055,5.838616,11.462640
std,0.063194,0.051508,0.052590,0.045229,0.049982,0.05433,0.045294,0.053131,0.052271,0.083289,...,1101.986319,852.463873,1026.612639,2.567621,2.253411,1.761138,2.948891,2.635710,2.948891,2.552572
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,150.000000,141.180000,125.000000,0.010000,0.010000,0.020000,0.540000,0.020000,0.540000,0.150000
25%,0.070000,0.080000,0.080000,0.080000,0.080000,0.08000,0.080000,0.080000,0.080000,0.050000,...,1090.180000,550.000000,1225.000000,2.660000,2.590000,1.860000,3.450000,3.410000,3.450000,10.200000
50%,0.100000,0.100000,0.100000,0.100000,0.100000,0.10000,0.100000,0.100000,0.100000,0.090000,...,1750.000000,902.500000,1900.000000,4.300000,3.240000,2.740000,5.320000,4.190000,5.320000,11.010000
75%,0.120000,0.120000,0.120000,0.120000,0.120000,0.12000,0.120000,0.120000,0.120000,0.110000,...,2666.670000,1569.167500,2583.330000,7.180000,5.260000,3.990000,8.250000,7.100000,8.250000,12.250000
max,0.940000,0.750000,0.670000,0.590000,0.700000,0.60000,0.480000,0.540000,0.680000,0.740000,...,4850.000000,4725.000000,4875.000000,10.260000,9.200000,8.620000,12.080000,11.070000,12.080000,18.200000


# What is the distribution of model certainty?
What does certainty/uncertainty look like when we are wrong as oppose to when we are right?

### Investigating with log probability

In [ ]:
rf_results = []

all_probabilities = {}
wrong_probabilities = {}
correct_probabilities = {}

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for user_id in wisdm.user_ids:
        test_set = wisdm.get_user_set(user_id)
        test_set = wisdm.remove_all_nan(test_set)

        test_labels = np.array([t.decode("utf-8") for t in test_set['class'].as_matrix()])
        test_features = test_set.as_matrix(columns=[test_set.columns[1:-1]])

        #print("%s labels, %s features" % (len(test_labels), len(test_features)))
        # training features & labels
        training_set = wisdm.data_df[wisdm.data_df['user'] != user_id]
        training_set = wisdm.remove_all_nan(training_set)
        training_labels = np.array([t.decode("utf-8") for t in training_set['class'].as_matrix()])
        training_features = training_set.as_matrix(columns=[test_set.columns[1:-1]])
        
        try:
            # normalize features
            scaler = StandardScaler().fit(training_features)
            scaled_train_x = scaler.transform(training_features)
            scaled_test_x = scaler.transform(test_features)

            clf = wisdm.weka_RF()
            clf.fit(scaled_train_x, training_labels)
            prediction_probs = clf.predict_log_proba(scaled_test_x)

            score = accuracy_score(test_labels, predictions)
            print("Score for %s: %.3f" % (user_id, score))
            rf_results.append(score)
        except ValueError as ve:
            if "while a minimum of 1 is required by StandardScaler" in ve.args[0]:
                print("Not enough data for user #%s" % user_id)
                continue
            else:
                raise ve

print("RF results : M=%.5f, SD=%.5f" % (np.mean(rf_results), np.std(rf_results)))

# Test Universal Impersonal Model Pipeline

In [3]:
start = time.time()

In [ ]:
rf_results = []

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for user_id in wisdm.user_ids:
        test_set = wisdm.get_user_set(user_id)
        test_set = wisdm.remove_all_nan(test_set)

        test_labels = np.array([t.decode("utf-8") for t in test_set['class'].as_matrix()])
        test_features = test_set.as_matrix(columns=[test_set.columns[1:-1]])

        #print("%s labels, %s features" % (len(test_labels), len(test_features)))
        # training features & labels
        training_set = wisdm.data_df[wisdm.data_df['user'] != user_id]
        training_set = wisdm.remove_all_nan(training_set)
        training_labels = np.array([t.decode("utf-8") for t in training_set['class'].as_matrix()])
        training_features = training_set.as_matrix(columns=[test_set.columns[1:-1]])
        
        try:
            # normalize features
            scaler = StandardScaler().fit(training_features)
            scaled_train_x = scaler.transform(training_features)
            scaled_test_x = scaler.transform(test_features)

            clf = wisdm.weka_RF()
            clf.fit(scaled_train_x, training_labels)
            predictions = clf.predict(scaled_test_x)

            score = accuracy_score(test_labels, predictions)
            print("Score for %s: %.3f" % (user_id, score))
            rf_results.append(score)
        except ValueError as ve:
            if "while a minimum of 1 is required by StandardScaler" in ve.args[0]:
                print("Not enough data for user #%s" % user_id)
                continue
            else:
                raise ve

print("RF results : M=%.5f, SD=%.5f" % (np.mean(rf_results), np.std(rf_results)))

In [ ]:
finish = time.time()
print("Took about %s seconds" % (finish - start))

# Setup Parallelization

In [4]:
import ipyparallel as ipp

In [5]:
c = ipp.Client()

In [6]:
dview = c[:]

In [7]:
%%px --local
import sys
sys.path.append("/home/sac086/wisdm_model_personalization/")
import warnings
import os
from wisdm import wisdm
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.cluster import KMeans
from scipy.stats import mode
from collections import Counter
import time

In [8]:
%%px --local

wisdm.set_data(version="2")

# Cross-Validation with exact number of data points

### Experiment Attributes : 
* amount of training set from individual end user
* amount of training set from impersonal data (other users)
    * "ALL" all other data
    * "closest cluster" only data from the closest cluster
    * "furthest cluster" only data from the furthest cluster
    * "All - furthest cluster" all other data EXCEPT data from furthest cluster
* test user id
* algorithm
* algorithm parameters



In [9]:
%%px --local
def personal_model(active_features, active_labels, test_features, test_labels):
    scaler = StandardScaler().fit(active_features)
    scaled_train_x = scaler.transform(active_features)
    scaled_test_x = scaler.transform(test_features)

    rfc_clf = wisdm.weka_RF()
    rfc_clf.fit(scaled_train_x, active_labels)
    predictions = rfc_clf.predict(scaled_test_x)
    score = accuracy_score(test_labels, predictions)
    return score

In [10]:
%%px --local

# Rewrite these so that if there are no test_features or labels, 
# then return the fit model itself
def universal_model(universal_features, universal_labels, test_features, test_labels):
    scaler = StandardScaler().fit(universal_features)
    scaled_train_x = scaler.transform(universal_features)
    scaled_test_x = scaler.transform(test_features)

    rfc_clf = wisdm.weka_RF()
    rfc_clf.fit(scaled_train_x, universal_labels)
    predictions = rfc_clf.predict(scaled_test_x)
    score = accuracy_score(test_labels, predictions)

    return score

In [11]:
%%px --local

def universal_plus_personal_model(personal_features, personal_labels,
                                  universal_features, universal_labels,
                                  test_features, test_labels):
    personal_plus_universal_features = np.vstack((personal_features, universal_features))
    personal_plus_universal_labels = np.hstack((personal_labels, universal_labels))

    scaler = StandardScaler().fit(personal_plus_universal_features)
    scaled_train_x = scaler.transform(personal_plus_universal_features)
    scaled_test_x = scaler.transform(test_features)

    rfc_clf = wisdm.weka_RF()

    rfc_clf.fit(scaled_train_x, personal_plus_universal_labels)
    predictions = rfc_clf.predict(scaled_test_x)
    score = accuracy_score(test_labels, predictions)
    return score

In [50]:
%%px --local

def cluster_plus_personal_model(personal_features, personal_labels,
                                  universal_features, universal_labels,
                                  test_features, test_labels, KM, clusters):
    cluster_predictions = KM.predict(personal_features)
    closest_cluster = mode(cluster_predictions).mode[0]

    cluster_data_indeces = [i for i in range(len(clusters)) if clusters[i] == closest_cluster]
    cluster_features = universal_features[cluster_data_indeces]
    cluster_labels = universal_labels[cluster_data_indeces]

    training_features = np.vstack((personal_features, cluster_features))
    training_labels = np.hstack((personal_labels, cluster_labels))

    scaler = StandardScaler().fit(training_features)
    scaled_train_x = scaler.transform(training_features)
    scaled_test_x = scaler.transform(test_features)

    rfc_clf = wisdm.weka_RF()

    rfc_clf.fit(scaled_train_x, training_labels)
    predictions = rfc_clf.predict(scaled_test_x)
    score = accuracy_score(test_labels, predictions)
    return score

In [ ]:
%%px --local
experiment_output_path = "/home/sac086/wisdm_model_personalization/results/experiment_08-24_train_v2/"
if not os.path.exists(experiment_output_path):
    os.makedirs(experiment_output_path)

In [18]:
%%px --local
training_sizes = [10,20,30,40,50,60,70,80,90,100]

def all_models():
    scores = []
    err = None
    training_sizes = [10,20,30,40,50,60,70,80,90,100]

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for ind, user_id in enumerate(user_ids):
            user_scores_df = []
            print("Running user #%s: %s" % (ind, user_id))
            personal_set = wisdm.get_user_set(user_id)
            personal_set = wisdm.remove_all_nan(personal_set)
            
            if len(personal_set) < 40:
                print("User does not have enough labeled data")
                continue

            personal_labels = np.array([t.decode("utf-8") for t in personal_set['class'].as_matrix()])
            personal_features = personal_set.as_matrix(columns=[personal_set.columns[1:-1]])

            # What is the distribution of labels for this participant?
            personal_labels_distribution = Counter(personal_labels)
            print("\tHas %s labels : " % len(personal_labels))
            for label_key, number in personal_labels_distribution.items():
                print("\t\t%s:%s" % (label_key, number))
            print("\n")
            # training features & labels
            universal_set = wisdm.data_df[wisdm.data_df['user'] != user_id]
            universal_set = wisdm.remove_all_nan(universal_set)
            universal_labels = np.array([t.decode("utf-8") for t in universal_set['class'].as_matrix()])
            universal_features = universal_set.as_matrix(columns=[universal_set.columns[1:-1]])

            # get k-means clusters
            number_of_clusters = 4 # the higher this number is, the smaller we should expect each cluster to be

            KM = KMeans(n_clusters=number_of_clusters)
            clusters = KM.fit_predict(universal_features)
            k = 10

            skf = StratifiedKFold(n_splits=k)

            k_run = 0
            try:
                for active_index, test_index in skf.split(personal_features, personal_labels):
                    print("\tRunning Fold #%s\n" % k_run)
                    # data set available for active labeling from the individual
                    all_active_features = personal_features[active_index]
                    all_active_labels = personal_labels[active_index]


                    # held out test set from individual
                    test_features = personal_features[test_index]
                    test_labels = personal_labels[test_index]

                    # iterate through size of training data
                    for training_size in training_sizes:
                        # initialize score holders
                        personal_model_scores = []
                        universal_model_scores = []
                        personal_plus_all_scores = []
                        personal_plus_cluster_scores = []

                        # run universal model
                        universal_model_score = universal_model(universal_features, universal_labels,
                                                                test_features, test_labels)
                        universal_model_scores.append(universal_model_score)

                        sss = StratifiedShuffleSplit(n_splits=5, train_size=training_size)

                        splits = sss.split(all_active_features, all_active_labels)

                        try:
                            for split_num, split_tup in enumerate(splits):
                                sampled_active_index, __ = split_tup
                                sampled_active_features = personal_features[sampled_active_index]
                                sampled_active_labels = personal_labels[sampled_active_index]

                                # run personal model
                                personal_score = personal_model(sampled_active_features, sampled_active_labels, test_features, test_labels)
                                personal_model_scores.append(personal_score)

                                # run personal + universal
                                personal_plus_all_score = universal_plus_personal_model(sampled_active_features, sampled_active_labels,
                                                                                        universal_features, universal_labels,
                                                                                        test_features, test_labels)
                                personal_plus_all_scores.append(personal_plus_all_score)

                                # run personal + cluster
                                personal_plus_cluster_score = cluster_plus_personal_model(sampled_active_features, sampled_active_labels,
                                                                                        universal_features, universal_labels,
                                                                                        test_features, test_labels, KM, clusters)
                                personal_plus_cluster_scores.append(personal_plus_cluster_score)
                        except ValueError as ve:
                            print("Error with training size while trying to split personal data")
                            print("Message : %s" % ve.args[0])
                            err = ve
                            if "Reduce test_size and/or train_size" in ve.args[0]:
                                print("continuing...")
                                continue
                            elif "should be smaller than the number of samples" in ve.args[0]:
                                print("continuing...")
                                continue
                            elif "The least populated class in y has only 1 member" in ve.args[0]:
                                print("continuing...")
                                continue
                            else:
                                raise(ve)

                        row = {"test user" : user_id,
                               "k-run" : k_run,
                           "classifier" : "RF with Wiki Parameters",
                           "personal training data" : training_size,
                           "personal score Mean" : np.mean(personal_model_scores),
                           "personal score STD" : np.std(personal_model_scores),
                           "impersonal score Mean" : np.mean(universal_model_scores),
                           "impersonal score STD" : np.std(universal_model_scores),
                           "personal + impersonal score Mean" : np.mean(personal_plus_all_scores),
                           "personal + impersonal score STD" : np.std(personal_plus_all_scores),
                           "personal + cluster score Mean" : np.mean(personal_plus_cluster_scores),
                           "personal + cluster score STD" : np.std(personal_plus_cluster_scores)
                           }
                        print("\tamount of personal data : %s row" % training_size)
                        print("\tpersonal model score : M=%.3f, SD=%.3f" % (row["personal score Mean"], row["personal score STD"]))
                        print("\tuniversal model score : M=%.3f, SD=%.3f" % (row["impersonal score Mean"], row["impersonal score STD"]))
                        print("\tpersonal + ALL universal : M=%.3f, SD=%.3f" % (row["personal + impersonal score Mean"], row["personal + impersonal score STD"]))
                        print("\tpersonal + CLUSTER universal : M=%.3f, SD=%.3f" % (row["personal + cluster score Mean"], row["personal + cluster score STD"]))
                        print("\n")
                        scores.append(row)
                        user_scores_df.append(row)
                    k_run += 1
            except ValueError as ve:
                if "Cannot have number of splits n_splits" in ve.args[0]:
                    print("Skipping this k-fold because there is not enough data...")
                    continue
                else:
                    raise ve
            user_scores_df = pd.DataFrame(user_scores_df)
            user_scores_df.to_pickle("/home/sac086/wisdm_model_personalization/results/experiment_08-24_v2_dataset/"+user_id+".pickle")

    scores_df = pd.DataFrame(scores)

In [13]:
scatter_result = dview.scatter("user_ids", wisdm.user_ids)

In [ ]:
start = time.time()
dview.block = True
results = dview.execute("all_models()")
finish = time.time()
print("Finished all models in %s seconds" % (finish - start))

In [ ]:
results.display_outputs()

# Train on v1.1, run on v2.0

In [14]:
%%px --local
experiment_output_path = "/home/sac086/wisdm_model_personalization/results/experiment_08-24_train_v1_test_v2/"
if not os.path.exists(experiment_output_path):
    os.makedirs(experiment_output_path)

In [51]:
%%px --local
training_sizes = [10,20,30,40,50,60,70,80,90,100]
k = 10

def all_models():
    scores = []
    err = None
    training_sizes = [10,20,30,40,50,60,70,80,90,100]
    
    # Train model with v1.1 data and get clusterings
    wisdm.set_data(version='1', make_compatible=True)
    
    data_df_v1 = wisdm.remove_all_nan(wisdm.data_df)
    user_ids_v1 = wisdm.user_ids
    
    impersonal_labels = np.array([t.decode("utf-8") for t in data_df_v1['class'].as_matrix()])
    impersonal_features = data_df_v1.as_matrix(columns=[data_df_v1.columns[1:-1]])
    
    # train an impersonal model
    impersonal_scaler = StandardScaler().fit(impersonal_features)
    scaled_train_x = impersonal_scaler.transform(impersonal_features)

    rfc_clf = wisdm.weka_RF()
    rfc_clf.fit(scaled_train_x, impersonal_labels)
    
    # create clusters
    number_of_clusters = 4 # the higher this number is, the smaller we should expect each cluster to be

    KM = KMeans(n_clusters=number_of_clusters)
    clusters = KM.fit_predict(scaled_train_x) # NOTE!!! <--- Really not sure if I should scale these before clustering
    
    # reset data back to v2.0
    wisdm.set_data(version="2", make_compatible=True)
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for ind, user_id in enumerate(user_ids):
            user_scores_df = []
            print("Running user #%s: %s" % (ind, user_id))
            personal_set = wisdm.get_user_set(user_id)
            personal_set = wisdm.remove_all_nan(personal_set)
            print("%s personal samples" % len(personal_set))

            personal_labels = np.array([t.decode("utf-8") for t in personal_set['class'].as_matrix()])
            personal_features = personal_set.as_matrix(columns=[personal_set.columns[1:-1]])

            # What is the distribution of labels for this participant?
            #personal_labels_distribution = Counter(personal_labels)
            #print("\tHas %s labels : " % len(personal_labels))
            #for label_key, number in personal_labels_distribution.items():
            #    print("\t\t%s:%s" % (label_key, number))
            #print("\n")
            
            skf = StratifiedKFold(n_splits=k)

            k_run = 0
            try:
                for active_index, test_index in skf.split(personal_features, personal_labels):
                    #print("\tRunning Fold #%s\n" % k_run)
                    # data set available for active labeling from the individual
                    all_active_features = personal_features[active_index]
                    all_active_labels = personal_labels[active_index]


                    # held out test set from individual
                    test_features = personal_features[test_index]
                    test_labels = personal_labels[test_index]

                    # iterate through size of training data
                    for training_size in training_sizes:
                        # initialize score holders
                        personal_model_scores = []
                        universal_model_scores = []
                        personal_plus_all_scores = []
                        personal_plus_cluster_scores = []

                        # run universal model
                        impersonal_scaled_test_x = impersonal_scaler.transform(test_features)
                        #universal_model_score = universal_model(universal_features, universal_labels,
                        #                                        impersonal_scaled_test_x, test_labels)
                        universal_model_score = accuracy_score(test_labels, rfc_clf.predict(impersonal_scaled_test_x))
                        universal_model_scores.append(universal_model_score)

                        sss = StratifiedShuffleSplit(n_splits=5, train_size=training_size)

                        splits = sss.split(all_active_features, all_active_labels)

                        try:
                            for split_num, split_tup in enumerate(splits):
                                sampled_active_index, __ = split_tup
                                sampled_active_features = personal_features[sampled_active_index]
                                sampled_active_labels = personal_labels[sampled_active_index]

                                # run personal model
                                personal_score = personal_model(sampled_active_features, sampled_active_labels, test_features, test_labels)
                                personal_model_scores.append(personal_score)

                                # run personal + universal
                                personal_plus_all_score = universal_plus_personal_model(sampled_active_features, sampled_active_labels,
                                                                                        impersonal_features, impersonal_labels,
                                                                                        test_features, test_labels)
                                personal_plus_all_scores.append(personal_plus_all_score)

                                # run personal + cluster
                                personal_plus_cluster_score = cluster_plus_personal_model(sampled_active_features, sampled_active_labels,
                                                                                        impersonal_features, impersonal_labels,
                                                                                        test_features, test_labels, KM, clusters)
                                personal_plus_cluster_scores.append(personal_plus_cluster_score)
                        except ValueError as ve:
                            print("Error with training size while trying to split personal data")
                            print("Message : %s" % ve.args[0])
                            err = ve
                            if "Reduce test_size and/or train_size" in ve.args[0]:
                                print("continuing...")
                                continue
                            elif "should be smaller than the number of samples" in ve.args[0]:
                                print("continuing...")
                                continue
                            elif "The least populated class in y has only 1 member" in ve.args[0]:
                                print("continuing...")
                                continue
                            else:
                                raise(ve)

                        row = {"test user" : user_id,
                               "k-run" : k_run,
                           "classifier" : "RF with Wiki Parameters",
                           "personal training data" : training_size,
                           "personal score Mean" : np.mean(personal_model_scores),
                           "personal score STD" : np.std(personal_model_scores),
                           "impersonal score Mean" : np.mean(universal_model_scores),
                           "impersonal score STD" : np.std(universal_model_scores),
                           "personal + impersonal score Mean" : np.mean(personal_plus_all_scores),
                           "personal + impersonal score STD" : np.std(personal_plus_all_scores),
                           "personal + cluster score Mean" : np.mean(personal_plus_cluster_scores),
                           "personal + cluster score STD" : np.std(personal_plus_cluster_scores)
                           }
                        print("\tamount of personal data : %s row" % training_size)
                        print("\tpersonal model score : M=%.3f, SD=%.3f" % (row["personal score Mean"], row["personal score STD"]))
                        print("\tuniversal model score : M=%.3f, SD=%.3f" % (row["impersonal score Mean"], row["impersonal score STD"]))
                        print("\tpersonal + ALL universal : M=%.3f, SD=%.3f" % (row["personal + impersonal score Mean"], row["personal + impersonal score STD"]))
                        print("\tpersonal + CLUSTER universal : M=%.3f, SD=%.3f" % (row["personal + cluster score Mean"], row["personal + cluster score STD"]))
                        print("\n")
                        scores.append(row)
                        user_scores_df.append(row)
                    k_run += 1
            except ValueError as ve:
                if "Cannot have number of splits n_splits" in ve.args[0]:
                    print("Skipping this k-fold (%s) because there is not enough data..." % k_run)
                    continue
                elif "while a minimum of 1 is required" in ve.args[0]:
                    print("Skipping this k-fold because there is not enough data...")
                    continue
                else:
                    raise ve
            user_scores_df = pd.DataFrame(user_scores_df)
            user_scores_df.to_pickle(experiment_output_path+user_id+".pickle")

    scores_df = pd.DataFrame(scores)

In [52]:
all_models()

Running user #0: 194
168 personal samples
	amount of personal data : 10 row
	personal model score : M=0.779, SD=0.117
	universal model score : M=0.789, SD=0.000
	personal + ALL universal : M=0.989, SD=0.021
	personal + CLUSTER universal : M=0.947, SD=0.082


	amount of personal data : 20 row
	personal model score : M=0.916, SD=0.026
	universal model score : M=0.789, SD=0.000
	personal + ALL universal : M=0.958, SD=0.021
	personal + CLUSTER universal : M=0.916, SD=0.026


	amount of personal data : 30 row
	personal model score : M=0.937, SD=0.039
	universal model score : M=0.789, SD=0.000
	personal + ALL universal : M=0.989, SD=0.021
	personal + CLUSTER universal : M=0.937, SD=0.039


	amount of personal data : 40 row
	personal model score : M=0.947, SD=0.000
	universal model score : M=0.789, SD=0.000
	personal + ALL universal : M=0.979, SD=0.026
	personal + CLUSTER universal : M=0.947, SD=0.033


	amount of personal data : 50 row
	personal model score : M=0.968, SD=0.042
	universal mod

	amount of personal data : 90 row
	personal model score : M=0.988, SD=0.024
	universal model score : M=0.765, SD=0.000
	personal + ALL universal : M=0.988, SD=0.024
	personal + CLUSTER universal : M=0.988, SD=0.024


	amount of personal data : 100 row
	personal model score : M=0.976, SD=0.029
	universal model score : M=0.765, SD=0.000
	personal + ALL universal : M=0.976, SD=0.029
	personal + CLUSTER universal : M=0.976, SD=0.029


	amount of personal data : 10 row
	personal model score : M=0.918, SD=0.080
	universal model score : M=0.647, SD=0.000
	personal + ALL universal : M=0.965, SD=0.029
	personal + CLUSTER universal : M=0.941, SD=0.037


	amount of personal data : 20 row
	personal model score : M=0.906, SD=0.047
	universal model score : M=0.647, SD=0.000
	personal + ALL universal : M=0.953, SD=0.069
	personal + CLUSTER universal : M=0.929, SD=0.069


	amount of personal data : 30 row
	personal model score : M=0.965, SD=0.029
	universal model score : M=0.647, SD=0.000
	personal + 

	amount of personal data : 70 row
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.375, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 80 row
	personal model score : M=0.988, SD=0.025
	universal model score : M=0.375, SD=0.000
	personal + ALL universal : M=0.988, SD=0.025
	personal + CLUSTER universal : M=0.988, SD=0.025


	amount of personal data : 90 row
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.375, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 100 row
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.375, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 10 row
	personal model score : M=0.738, SD=0.139
	universal model score : M=0.375, SD=0.000
	personal + 

	amount of personal data : 50 row
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.000, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 60 row
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.000, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 70 row
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.000, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 80 row
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.000, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 90 row
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.000, SD=0.000
	personal + A

	amount of personal data : 30 row
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.000, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 40 row
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.000, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 50 row
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.000, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 60 row
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.000, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 70 row
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.000, SD=0.000
	personal + A

	amount of personal data : 10 row
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.000, SD=0.000
	personal + ALL universal : M=0.981, SD=0.039
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 20 row
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.000, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 30 row
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.000, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 40 row
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.000, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 50 row
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.000, SD=0.000
	personal + A

In [20]:
scatter_result = dview.scatter("user_ids", wisdm.user_ids)

In [21]:
start = time.time()
dview.block = True
results = dview.execute("all_models()")
finish = time.time()
print("Finished all models in %s minutes" % ((finish - start) / 60.)

Finished all models in 1690.7465000152588 seconds


In [22]:
results.display_outputs()

[stdout:0] 
Running user #0: 194
168 personal samples
	amount of personal data : 10 row
	personal model score : M=0.884, SD=0.070
	universal model score : M=0.737, SD=0.000
	personal + ALL universal : M=0.979, SD=0.026
	personal + CLUSTER universal : M=0.968, SD=0.026


	amount of personal data : 20 row
	personal model score : M=0.895, SD=0.033
	universal model score : M=0.737, SD=0.000
	personal + ALL universal : M=0.958, SD=0.021
	personal + CLUSTER universal : M=0.916, SD=0.026


	amount of personal data : 30 row
	personal model score : M=0.905, SD=0.021
	universal model score : M=0.737, SD=0.000
	personal + ALL universal : M=0.958, SD=0.039
	personal + CLUSTER universal : M=0.926, SD=0.042


	amount of personal data : 40 row
	personal model score : M=0.958, SD=0.039
	universal model score : M=0.737, SD=0.000
	personal + ALL universal : M=0.979, SD=0.026
	personal + CLUSTER universal : M=0.947, SD=0.033


	amount of personal data : 50 row
	personal model score : M=0.937, SD=0.039
	u

#  Active Learning Experiments (Uncertainty Sampling with Random Forest)

Ideas for experiments:
1. Simply vary the uncertainty threshold to see how this affects the number of labels elicited and the result accuracies
2. Same as #1 but incorporate samples with label of certainty into training set.
3. Same as #2, but also propogate labels to unlabeled sample set and incorporate into training

### Active Learning Experiment 1

In [54]:
%%px --local
experiment_output_path = "/home/sac086/wisdm_model_personalization/results/experiment_08-28_train_v1_test_v2_active1/"
if not os.path.exists(experiment_output_path):
    os.makedirs(experiment_output_path)

In [55]:
def uncertainty_sampling(features, model, uncertainty_threshold):
    '''returns two-tuple with predictions as the first element and
       indeces of uncertain predictions as the second'''
    prediction_probabilities = model.predict_proba(features)
    thresh = model.n_classes_ * uncertainty_threshold 
    uncertain_indeces = [i for i in range(len(prediction_probabilities)) \
                         if np.sum(prediction_probabilities[i]) < thresh]
    
    return model.predict(features), uncertain_indeces

In [86]:
%%px --local
#training_sizes = [10,20,30,40,50,60,70,80,90,100]
uncertainty_thresholds = [.1,.2,.3,.4,.5,.6,.7,.8,.9]
k = 10

def active_learning_experiment1():
    scores = []
    err = None
    training_sizes = [10,20,30,40,50,60,70,80,90,100]
    
    # Train model with v1.1 data and get clusterings
    wisdm.set_data(version='1', make_compatible=True)
    
    data_df_v1 = wisdm.remove_all_nan(wisdm.data_df)
    user_ids_v1 = wisdm.user_ids
    
    impersonal_labels = np.array([t.decode("utf-8") for t in data_df_v1['class'].as_matrix()])
    impersonal_features = data_df_v1.as_matrix(columns=[data_df_v1.columns[1:-1]])
    
    # train an impersonal model
    impersonal_scaler = StandardScaler().fit(impersonal_features)
    scaled_train_x = impersonal_scaler.transform(impersonal_features)

    rfc_clf = wisdm.weka_RF()
    rfc_clf.fit(scaled_train_x, impersonal_labels)
    
    # create clusters
    number_of_clusters = 4 # the higher this number is, the smaller we should expect each cluster to be

    KM = KMeans(n_clusters=number_of_clusters)
    clusters = KM.fit_predict(scaled_train_x) # NOTE!!! <--- Really not sure if I should scale these before clustering
    
    # reset data back to v2.0
    wisdm.set_data(version="2", make_compatible=True)
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for ind, user_id in enumerate(user_ids):
            user_scores_df = []
            print("Running user #%s: %s" % (ind, user_id))
            personal_set = wisdm.get_user_set(user_id)
            personal_set = wisdm.remove_all_nan(personal_set)
            print("%s personal samples" % len(personal_set))

            personal_labels = np.array([t.decode("utf-8") for t in personal_set['class'].as_matrix()])
            personal_features = personal_set.as_matrix(columns=[personal_set.columns[1:-1]])

            # What is the distribution of labels for this participant?
            #personal_labels_distribution = Counter(personal_labels)
            #print("\tHas %s labels : " % len(personal_labels))
            #for label_key, number in personal_labels_distribution.items():
            #    print("\t\t%s:%s" % (label_key, number))
            #print("\n")
            
            skf = StratifiedKFold(n_splits=k)

            k_run = 0
            for active_index, test_index in skf.split(personal_features, personal_labels):
                #print("\tRunning Fold #%s\n" % k_run)
                # data set available for active labeling from the individual
                all_active_features = personal_features[active_index]
                all_active_labels = personal_labels[active_index]


                # held out test set from individual
                test_features = personal_features[test_index]
                test_labels = personal_labels[test_index]

                personal_model_scores = []
                universal_model_scores = []
                personal_plus_all_scores = []
                personal_plus_cluster_scores = []

                # run universal model
                impersonal_scaled_test_x = impersonal_scaler.transform(test_features)
                universal_model_score = accuracy_score(test_labels, rfc_clf.predict(impersonal_scaled_test_x))
                universal_model_scores.append(universal_model_score)

                for thresh in uncertainty_thresholds:
                    # determine active samples from uncertainty sampling
                    predictions, sampled_active_index = uncertainty_sampling(all_active_features, rfc_clf, thresh)
                    sampled_active_features = personal_features[sampled_active_index]
                    sampled_active_labels = personal_labels[sampled_active_index]
                    if len(sampled_active_labels) < 1:
                        row = {"test user" : user_id,
                           "k-run" : k_run,
                       "classifier" : "RF with Wiki Parameters",
                       "personal training data" : len(sampled_active_index),
                       "uncertainty threshold" : thresh,
                       "personal score Mean" : np.nan,
                       "personal score STD" : np.nan,
                       "impersonal score Mean" : np.nan,
                       "impersonal score STD" : np.nan,
                       "personal + impersonal score Mean" :np.nan,
                       "personal + impersonal score STD" : np.nan,
                       "personal + cluster score Mean" : np.nan,
                       "personal + cluster score STD" : np.nan
                       }
                        scores.append(row)
                        user_scores_df.append(row)
                        continue
                    #else:
                        #return sampled_active_features, sampled_active_labels, impersonal_features, impersonal_labels, test_features, test_labels, KM, clusters


                    # make predictions
                    personal_score = personal_model(sampled_active_features, sampled_active_labels, test_features, test_labels)
                    personal_model_scores.append(personal_score)

                    # run personal + universal
                    personal_plus_all_score = universal_plus_personal_model(sampled_active_features, sampled_active_labels,
                                                                            impersonal_features, impersonal_labels,
                                                                            test_features, test_labels)
                    personal_plus_all_scores.append(personal_plus_all_score)

                    # run personal + cluster
                    personal_plus_cluster_score = cluster_plus_personal_model(sampled_active_features, sampled_active_labels,
                                                                            impersonal_features, impersonal_labels,
                                                                            test_features, test_labels, KM, clusters)
                    personal_plus_cluster_scores.append(personal_plus_cluster_score)
                    row = {"test user" : user_id,
                           "k-run" : k_run,
                       "classifier" : "RF with Wiki Parameters",
                       "personal training data" : len(sampled_active_index),
                       "uncertainty threshold" : thresh,
                       "personal score Mean" : np.mean(personal_model_scores),
                       "personal score STD" : np.std(personal_model_scores),
                       "impersonal score Mean" : np.mean(universal_model_scores),
                       "impersonal score STD" : np.std(universal_model_scores),
                       "personal + impersonal score Mean" : np.mean(personal_plus_all_scores),
                       "personal + impersonal score STD" : np.std(personal_plus_all_scores),
                       "personal + cluster score Mean" : np.mean(personal_plus_cluster_scores),
                       "personal + cluster score STD" : np.std(personal_plus_cluster_scores)
                       }
                    print("\tamount of personal data : %s samples" % len(sampled_active_index))
                    print("\tUncertainty threshold : %s" % thresh)
                    print("\tpersonal model score : M=%.3f, SD=%.3f" % (row["personal score Mean"], row["personal score STD"]))
                    print("\tuniversal model score : M=%.3f, SD=%.3f" % (row["impersonal score Mean"], row["impersonal score STD"]))
                    print("\tpersonal + ALL universal : M=%.3f, SD=%.3f" % (row["personal + impersonal score Mean"], row["personal + impersonal score STD"]))
                    print("\tpersonal + CLUSTER universal : M=%.3f, SD=%.3f" % (row["personal + cluster score Mean"], row["personal + cluster score STD"]))
                    print("\n")
                    scores.append(row)
                    user_scores_df.append(row)
            k_run += 1
        user_scores_df = pd.DataFrame(user_scores_df)
        user_scores_df.to_pickle(experiment_output_path+user_id+".pickle")

In [67]:
scatter_result = dview.scatter("user_ids", wisdm.user_ids)

In [68]:
user_ids

array(['194', '998'], dtype=object)

In [ ]:
active_learning_experiment1()

Running user #0: 194
168 personal samples
	amount of personal data : 149 samples
	Uncertainty threshold : 0.3
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.895, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 149 samples
	Uncertainty threshold : 0.4
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.895, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 149 samples
	Uncertainty threshold : 0.5
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.895, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 149 samples
	Uncertainty threshold : 0.6
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.895, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	perso

	amount of personal data : 151 samples
	Uncertainty threshold : 0.8
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.765, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 151 samples
	Uncertainty threshold : 0.9
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.765, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 152 samples
	Uncertainty threshold : 0.3
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.625, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 152 samples
	Uncertainty threshold : 0.4
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.625, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.00

	amount of personal data : 153 samples
	Uncertainty threshold : 0.6
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.800, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 153 samples
	Uncertainty threshold : 0.7
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.800, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 153 samples
	Uncertainty threshold : 0.8
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.800, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.000


	amount of personal data : 153 samples
	Uncertainty threshold : 0.9
	personal model score : M=1.000, SD=0.000
	universal model score : M=0.800, SD=0.000
	personal + ALL universal : M=1.000, SD=0.000
	personal + CLUSTER universal : M=1.000, SD=0.00

In [85]:
sampled_active_features, sampled_active_labels, impersonal_features, impersonal_labels, test_features, test_labels, KM, clusters = active_learning_experiment1()

Running user #0: 194
168 personal samples


In [81]:
personal_plus_cluster_score = cluster_plus_personal_model(sampled_active_features, sampled_active_labels,
                                                                            impersonal_features, impersonal_labels,
                                                                            test_features, test_labels, KM, clusters)

In [83]:
np.mean(personal_plus_cluster_score)

0.84210526315789469

# Same experiments without stratified iterations (what happens with class imbalance?)